In [1]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

In [2]:
import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm

In [131]:
data = pd.read_pickle('집구하기(전처리).pkl')

## Doc2Vec

In [88]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from gensim.test.utils import common_texts

In [137]:
import ast
token = []
token_noun = []
for i in range(len(data['token'])):
    token.append(ast.literal_eval(data['token'][i]))
    token_noun.append(ast.literal_eval(data['text_noun'][i]))

In [139]:
data['token'] = token
data['text_noun'] = token_noun

In [144]:
documents = [TaggedDocument(word, [i]) for i, word in enumerate(data['token'])]

In [149]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=20,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=3,
                dm =1)
  
model.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=10)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [163]:
data['token'][0]

['그냥',
 '정도',
 '맨날',
 '정해진',
 '밥상',
 '고요',
 '집',
 '반찬',
 '다',
 '말',
 '근데',
 '요즘',
 '집',
 '밥',
 '먹다',
 '먹다',
 '뭐',
 '사기',
 '라푀',
 '유',
 '크로와상',
 '부산광역시',
 '부산진구',
 '중앙',
 '번가',
 '길',
 '층',
 '호',
 '라푀',
 '유',
 '크로와상',
 '서면',
 '크로와상',
 '맛집',
 '라푀',
 '유',
 '크로와상',
 '다',
 '들다',
 '알',
 '근데',
 '난',
 '원래',
 '어제',
 '빵',
 '사려',
 '유리창',
 '너머',
 '보이다',
 '녹차',
 '앙',
 '버터',
 '나르다',
 '유혹',
 '나다',
 '모르다',
 '들어가다',
 '사보',
 '림',
 '알바',
 '기전',
 '밥',
 '빵',
 '배',
 '채우다',
 '줍다',
 '막걸리',
 '저만큼',
 '말다',
 '먹다',
 '버리다',
 '쓰레기통',
 '버리다',
 '길',
 '버리다',
 '알바',
 '만들다',
 '먹다',
 '홀',
 '주먹밥',
 '인간',
 '배',
 '너무',
 '사실',
 '항상',
 '주다',
 '만',
 '주먹밥',
 '만',
 '들어주다',
 '동글동글',
 '말',
 '주다',
 '아침',
 '언니',
 '챙기다',
 '준',
 '바이오',
 '플레',
 '빨미까레',
 '먹어주다',
 '정도',
 '먹다',
 '출근',
 '맞다',
 '빨미까레',
 '근데',
 '빵집',
 '빨미까레',
 '흘리다',
 '말다',
 '직원',
 '새',
 '꺼내다',
 '차장',
 '아침',
 '불고기',
 '샌드위치',
 '만들다',
 '아가',
 '바나나',
 '우유',
 '주다',
 '점심',
 '다다믄',
 '만',
 '주문',
 '먹다',
 '오늘',
 '다앵',
 '점심',
 '충무김밥',
 '잔치국수',
 '원래',
 '충무김밥',
 '오징어',
 '제일',
 '알',
 '근데'

In [164]:
vector = model.infer_vector(data['token'][0])

In [165]:
len(data['token'])

27672

In [160]:
from tqdm import tqdm

In [167]:
model.random.seed(9999)
rv = []
for i in tqdm(data['token']):
    vector = model.infer_vector(i)
    rv.append(vector)

100%|███████████████████████████████████████████████████████████████████████████| 27672/27672 [00:59<00:00, 462.12it/s]


In [169]:
len(rv)

27672

In [170]:
data['review_vec'] = rv

In [178]:
data.rename(columns = {'Unnamed: 0': 'doc_num'}, inplace=True)

In [180]:
data.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1'], axis=1, inplace=True)

In [181]:
data

,doc_num,date,title,review,comment,doc,source,token,text_noun,review_vec
0,794,2022. 4. 12. 10:40,NaN,Diary in 2022 2022.04.03 - 2022.04...,[],그냥 이 정도면 맨날 정해진 밥상 왜냐고요 집에 반찬이 저게 다란 말이에여...,네이버뷰,"[그냥, 정도, 맨날, 정해진, 밥상, 고요, 집, 반찬, 다, 말, 근데, 요즘,...","[그냥, 정도, 정해진, 밥상, 고요, 집, 반찬, 말, 요즘, 집, 밥, 뭐, 사...","[-0.08329613, 0.011866811, 0.44026828, 0.48597..."
1,5916,2022. 4. 28. 19:01,NaN,E.E 220428 imsh ・ 2022. 4...,[],글 안 쓴 기간 동안 밀린 일상 일단 난 시험이 끝났다 일생연 빼고 중간 ...,네이버뷰,"[글, 쓸다, 기간, 밀리다, 일상, 난, 시험, 끝나다, 일생, 연, 빼다, 중간...","[글, 기간, 일상, 난, 시험, 일생, 연, 중간, 나머지, 썩, 뭐, 공부, 결...","[0.49730548, -0.6956667, -0.15713012, 0.393049..."
2,6739,2018. 6. 1. 23:33,NaN,ＴＯＫＹＯ tokyo days 87~94 ...,[],도쿄워홀 도쿄여행 도쿄카페 도쿄맛집 신주쿠카페 키치죠지카페 키치죠지 신주쿠...,네이버뷰,"[도쿄, 워홀, 도쿄, 여행, 도쿄, 카페, 도쿄, 맛집, 신주쿠, 카페, 키치, ...","[도쿄, 워홀, 도쿄, 여행, 도쿄, 카페, 도쿄, 맛집, 신주쿠, 카페, 키치, ...","[-0.025188802, -0.06214978, 0.36642823, 0.7276..."
3,9108,2021. 7. 1. 2:58,NaN,diary Life’s a beach sees...,[],안 밀리고 돌아온 월 포스팅이번달엔 종강도 하고 여행도 다녀오고 수고 많았...,네이버뷰,"[밀리, 돌아오다, 포스팅, 달, 종강, 여행, 다녀오다, 수고, 벌써, 반, 이렇...","[밀리, 포스팅, 달, 종강, 여행, 수고, 벌써, 반, 방학, 목표, 꼭, 아부지...","[0.6014674, -0.019872742, 1.270496, 0.54509866..."
4,11216,2022. 5. 3. 16:20,NaN,daily 2022. 3. ddori ・ 20...,[],어제로 급한 업무가 해결되어서드디어 한 숨 돌릴 여유가 생겨 밀린 월기를 ...,네이버뷰,"[어제, 업무, 해결, 드디어, 숨, 돌리다, 여유, 생기다, 밀리다, 기르다, 써...","[어제, 업무, 해결, 숨, 여유, 첫, 날, 석화, 칼, 장비, 장, 불면, 후루...","[-0.4001867, -0.49134222, 0.6571016, 0.7585181..."
...,...,...,...,...,...,...,...,...,...,...
27667,2594,2018.11.22. 20:41,난방용으로 어떤게 좋을까요?,히터나 컨벡터를 사려고 고민중이에요.\n저렴한것들이 많더라구요.\n\n근데 막상 보...,[['저는 오늘 에스워머를 질렀어요! 이건 원적외선 나와서 가까이대도 위험하지않다고...,히터나 컨벡터를 사려고 고민중이에요 저렴한것들이 많더라구요 근데 막상 보니까 겨울엔...,네이버카페,"[히터, 컨벡터, 사려, 고민, 들이다, 근데, 막상, 보, 겨울, 기세, 무시, ...","[히터, 컨벡터, 사려, 고민, 막상, 보, 겨울, 기세, 무시, 침대, 방, 평,...","[0.109961875, 0.045498237, -0.10372741, 0.0872..."
27668,969,2020.11.05. 15:05,스타일러 쓰시는 분 단점 좀^^ 그리구요 히터와 온풍기의 차이가 무엇인가요?,히터는 석영관 데우는 거 빨갛게~\n온풍기는 바람이 나오는 거(홈쇼핑 욕실선전 그거...,"[['물통이 작다, 버리는물통과 배수관에 물때가 낀다..짜증나요 이거'], ['단점...",히터는 석영관 데우는 거 빨갛게 온풍기는 바람이 나오는 거 홈쇼핑 욕실선전 그거 히...,네이버카페,"[히터, 영관, 데우다, 풍기, 바람, 나오다, 홈쇼핑, 욕실, 선전, 그거, 히터...","[히터, 영관, 풍기, 바람, 홈쇼핑, 욕실, 선전, 그거, 히터, 호빵, 모양, ...","[-0.0037776246, -0.22041059, 0.07279901, 0.073..."
27669,151,2021.10.24. 07:46,열히터 vs 온풍기,히터랑 온풍기 중 어느게 더 가성비가 있을까요?\n둘이 용도가 어느정도 다른건 알고...,[],히터랑 온풍기 중 어느게 더 가성비가 있을까요 둘이 용도가 어느정도 다른건 알고 있...,네이버카페,"[히터, 풍기, 중, 더, 가성, 비, 용도, 정도, 알, 제품, 이면, 써다, 가...","[히터, 풍기, 중, 더, 가성, 비, 용도, 정도, 알, 제품, 이면, 가정, 전...","[0.0536199, -0.171595, -0.06380176, 0.09210086..."
27670,2066,2019.05.26. 18:50,미스터피자나 애슐리에서 샐러드바만 먹어도 괜찮나요?,힝 갑자기 땡겨서요ㅠㅠ\n미스터피자나 애슐리에서 샐러드바만 먹어도 괜찮나요?\n(혼...,[['미스터피자도 샐러드바만 주문이 가능한가요??? 애슐리는 샐러드바만 먹으러 가곤...,힝 갑자기 땡겨서요 미스터피자나 애슐리에서 샐러드바만 먹어도 괜찮나요 혼자 갈려구요...,네이버카페,"[힝, 갑자기, 땡기다, 미스터피자, 애슐리, 샐러드, 바, 먹다, 갈리다, 미스터...","[힝, 갑자기, 미스터피자, 애슐리, 샐러드, 바, 미스터피자, 샐러드, 바, 주문...","[0.025825458, -0.11067809, -0.05174353, 0.1080..."


In [183]:
data.to_pickle('team3_data_vectorized.pkl')